In [2]:
from bs4 import BeautifulSoup
import urllib.request

In [24]:
class BoxScoreEntry:
    def __init__(self, team, name, columns):
        
        self.team = team
        self.name = name
        
        self.minutes = columns[0].contents[0]
        
        self.field_goals_made = columns[1].contents[0]
        self.field_goals_attemped = columns[2].contents[0]

        self.threes_made = columns[4].contents[0]
        self.threes_attempted = columns[5].contents[0]

        self.free_throws_made = columns[7].contents[0]
        self.free_throws_attempted = columns[8].contents[0]

        self.offensive_rebounds = columns[10].contents[0]
        self.defensive_rebounds = columns[11].contents[0]
        
        self.assists = columns[13].contents[0]
        self.steals = columns[14].contents[0]
        self.blocks = columns[15].contents[0]
        self.turnovers = columns[16].contents[0]
        self.personal_fouls = columns[17].contents[0]
        self.points = columns[18].contents[0]

In [104]:
ref_url = "https://www.basketball-reference.com"

import calendar
import datetime

def get_games(dt):
    
    month = calendar.month_name[dt.month].lower()
    match = f"month={dt.month}&day={dt.day}&year={dt.year}"
    url = f"{ref_url}/leagues/NBA_{dt.year+1}_games-{month}.html"
    
    games = []
    
    with urllib.request.urlopen(url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        
        for table in soup.find_all("table"):
            if "id" not in table.attrs:
                continue
            
            if table["id"] != "schedule":
                continue
                
            skipped = False
            for tr in table.find_all("tr"):
                if not skipped:
                    skipped = True
                    continue
                
                href = tr.find_all("th")[0].find_all("a")[0]["href"]
                
                if match not in href:
                    continue
                    
                tds = tr.find_all("td")
                
                away = tds[1].find_all("a")[0].get_text()
                away_short = tds[1].find_all("a")[0]["href"].split('/')[2].lower()
                home = tds[3].find_all("a")[0].get_text()
                home_short = tds[3].find_all("a")[0]["href"].split('/')[2].lower()
                
                games.append((away_short, home_short))
                
    return games

In [105]:
dt = datetime.datetime.now()

get_games(dt)

[('mil', 'den'),
 ('cho', 'det'),
 ('ind', 'hou'),
 ('atl', 'lal'),
 ('orl', 'nyk'),
 ('bos', 'por')]

In [4]:
links = get_boxscore_links(2018,11,8)

In [5]:
with urllib.request.urlopen(f"{ref_url}/boxscores/201811080GSW.html") as response:
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')